In [26]:
        import Pkg
        Pkg.add("Conda")
        Pkg.add("PyCall")
        Pkg.build("PyCall")
        #Pkg.add("Distances")
        #Pkg.build("Distances")
        Pkg.add("NearestNeighbors")
        Pkg.add("Graphs")
        Pkg.add("DataFrames")
        Pkg.add("CSV")

        using Conda

        Conda.pip_interop(true)
        #Conda.pip("install", "scipy")
        #Conda.pip("install", "numpy")
        #Conda.pip("install", "leidenalg")

        using PyCall
        leidenalg = pyimport("leidenalg")
        igraph = pyimport("igraph")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/

PyObject <module 'igraph' from '/Users/lab/.julia/conda/3/lib/python3.10/site-packages/igraph/__init__.py'>

In [2]:
 using DelimitedFiles
#readdlm("/Users/lab/Documents/githubspot/deepmeltome/data/target_psmtable.txt")
#import Pkg; Pkg.add("CSV")
meltdata = readdlm("/Users/lab/Documents/githubspot/meltmatch/julia_clustering_input.csv",',', header=true)

([1.0 1.11025030862773 … 0.614151142101373 0.357674997935981; 1.0 0.889124231127228 … 0.27000521050942 0.248588426677347; … ; 1.0 0.817024811301291 … 0.499394661482848 0.208843314325353; 1.0 0.860524826231243 … 0.36773979751416 0.176085981594109], AbstractString["1" "2" … "7" "8"])

In [3]:
using Distances 

#Calculate the weighted Euclidean distance
function weighted_euclidean_distance(x::Array{Float64}, y::Array{Float64}, v::Int, N::Int, K::Int)
    sum(((x .- y).^2) .* v) / (N * K)
end

#Calculate the similarity matrix
function calculate_similarity(data::Array{Float64}, v::Int, N::Int, K::Int)
    n_obs = size(data, 1)
    similarity_matrix = Array{Float64}(undef, n_obs, n_obs)
    
    #Using the pairwise function from the Distances package for efficient distance computation
    pairwise_distances = pairwise(Euclidean(), data; dims=1)
    for i in 1:n_obs
        for j in 1:n_obs
            weighted_dist = sqrt(pairwise_distances[i, j] * v / (N * K))
            similarity_matrix[i, j] = 1 / (1 + weighted_dist)
        end
    end
    similarity_matrix
end


calculate_similarity (generic function with 1 method)

In [4]:
data_matrix, headers = meltdata
#similarity_matrix = calculate_similarity(data_matrix, 1, 1, 1) #unweighted

([1.0 1.11025030862773 … 0.614151142101373 0.357674997935981; 1.0 0.889124231127228 … 0.27000521050942 0.248588426677347; … ; 1.0 0.817024811301291 … 0.499394661482848 0.208843314325353; 1.0 0.860524826231243 … 0.36773979751416 0.176085981594109], AbstractString["1" "2" … "7" "8"])

In [15]:
using NearestNeighbors
using Graphs
using LinearAlgebra

#LSH-like function to approximate nearest neighbors
function approximate_neighbors(data::Array{Float64}, num_hashes::Int, bucket_size::Int)
    dim = size(data, 2)
    hash_functions = [randn(dim) for _ in 1:num_hashes]
    hashed_data = Array{Float64}(undef, size(data, 1), num_hashes) 

    for i in 1:size(data, 1)
        for j in 1:num_hashes
            hashed_data[i, j] = dot(data[i, :], hash_functions[j]) > 0 ? 1.0 : 0.0  
        end
    end

    #Create a nearest neighbors data structure
    tree = KDTree(transpose(hashed_data)) 

    #Find approximate nearest neighbors
    indices = [knn(tree, hashed_data[i, :], bucket_size, true)[1] for i in 1:size(hashed_data, 1)]

    return indices
end

#Function to create a graph from the neighbor indices
function create_graph(neighbor_indices)
    g = SimpleGraph(length(neighbor_indices))
    for (idx, neighbors) in enumerate(neighbor_indices)
        for neighbor in neighbors
            add_edge!(g, idx, neighbor)
        end
    end
    return g
end
#Function to create a graph - igraph
function create_igraph(neighbor_indices)
    edges = Vector{Tuple{Int64, Int64}}()
    for (idx, neighbors) in enumerate(neighbor_indices)
        for neighbor in neighbors
            push!(edges, (idx, neighbor))
        end
    end
    igraph_graph = igraph.Graph.TupleList(edges, directed=false)
    return igraph_graph
end


create_igraph (generic function with 1 method)

In [16]:
indices = approximate_neighbors(data_matrix, 10, 5)

57033377-element Vector{Vector{Int64}}:
 [46238641, 32286368, 32024212, 32288696, 38683988]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [13179087, 38687922, 38683980, 45013884, 13179091]
 [28325841, 17904992, 28505299, 77463, 109687]
 [7895702, 7910340, 8428670, 7734916, 7703622]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [46238641, 32286368, 32024212, 32288696, 38683988]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [46238641, 32286368, 32024212, 32288696, 38683988]
 [13179087, 38687922, 38683980, 45013884, 13179091]
 ⋮
 [13179087, 38687922, 38683980, 45013884, 13179091]
 [46238641, 32286368, 32024212, 32288696, 38683988]
 [7947539, 7947547, 7947538, 7947612, 7947526]
 [7947541, 7947552, 7947536, 7947551, 7948397]
 [7947541, 7947552, 7947536, 7947551, 7948397]
 [7948361, 7947542, 7948359, 7948356, 7948325]
 [7947539, 7947547, 7947538, 

In [17]:
graph = create_igraph(indices)
partition = leidenalg.find_partition(graph, leidenalg.ModularityVertexPartition)

PyObject <leidenalg.VertexPartition.ModularityVertexPartition object at 0x1916d3c10>

In [22]:
#Write files - graph
graph.write("julia_graph.graphml", "graphml")

In [27]:
#Write files - membership
using DataFrames
using CSV
membership = partition[:membership]
membership_df = DataFrame(vertex = 1:length(membership), cluster = membership)
CSV.write("partition_membership.csv", membership_df)

"partition_membership.csv"